# Explore OPM database with interactive query and graphs

## Package Requirements

In [2]:
import sys
import os
sys.path.append(os.path.abspath('..'))
import OPMxplore

['', 'C:\\python\\Anaconda', 'C:\\ProgramData\\Miniconda3\\python36.zip', 'C:\\ProgramData\\Miniconda3\\DLLs', 'C:\\ProgramData\\Miniconda3\\lib', 'C:\\ProgramData\\Miniconda3', 'C:\\ProgramData\\Miniconda3\\lib\\site-packages', 'C:\\ProgramData\\Miniconda3\\lib\\site-packages\\opmxplore-0.1.dev0-py3.6.egg', 'C:\\ProgramData\\Miniconda3\\lib\\site-packages\\pandasql-0.7.3-py3.6.egg', 'C:\\ProgramData\\Miniconda3\\lib\\site-packages\\win32', 'C:\\ProgramData\\Miniconda3\\lib\\site-packages\\win32\\lib', 'C:\\ProgramData\\Miniconda3\\lib\\site-packages\\Pythonwin', 'C:\\ProgramData\\Miniconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\T420\\.ipython']


In [2]:
%matplotlib inline 
%matplotlib notebook 


# Jupyter Notebook Formatting
from IPython.display import HTML

# import for OPM dataframe access
import os
import sys

#import package OPMxplore
sys.path.append(os.path.abspath('..'))
# alternate way to do the same:
# sys.path.append(os.path.dirname(os.getcwd()))
import OPMxplore
                        
# import for query [pypdb, sqlite3?, query?, pandasql?]
from pypdb import *
import pprint

# scipy stack
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd

# visualization stack [ipywidgets,nglview,matplotlib,seaborn,plotly]
import nglview as nv
from ipywidgets import interact

from plotly import __version__
print (__version__) # requires version >= 1.9.0

import plotly.offline as offline
# plotly.plotly.iplot() # online version
offline.init_notebook_mode(connected=True)    # inline 

import plotly.graph_objs as go
import cufflinks as cf
cf.go_offline() # cufflinks offline

import plotly.plotly as ply
import plotly
from plotly.widgets import GraphWidget as gw

# PDB parsing
from Bio.PDB import PDBParser 
from Bio.PDB import MMCIFParser
# from Bio.PDB import *

# Special offline API Setup Info

from pandasql import sqldf
sql_query = lambda q: sqldf(q, globals())

def make_sql(table,selection="*", options=""):
    return sql_query("SELECT "+selection+" FROM "+table+" "+options+";")

2.1.0


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [21]:
# testing package import
#package = os.path.join(os.path.dirname(os.getcwd()), 'OPMxplore')
#package
#!ls /Users/PnPofSila/CSE583/project/OPMxplore/OPMxplore

#from . import OPMxplore
# from package import *

# Step 1: Load OPM dataframe
## (currently has code, replace with wrapper)

In [9]:
def get_path(filename):
    """
    Locate the files in the appropriate directory relative to the current
    working directory. This assumes the current file is in
    "~/example_notebooks/", or in "~/pdb-search/"
    and that the data files are located in
    "~/pdb-search/data/sql_export/".
    
    os.path.join is used to provide cross-platform support.
    
    Returns:
    -------
    path : string
        The full path to the file
    """
    return os.path.join(os.path.dirname(os.getcwd()),
                        'OPMxplore',
                        'data',
                        'sql_export',
                        filename)

def load_data():
    """
    A Function to read all of the OPM database tables from csv files
    and load it into memory as a pandas dataframe.
    'protein.csv' contains the main table of protein information
    and the rest of the tables contain specific information about
    the various categories of proteins.  These are converted to dicts,
    and used to add the appropriate columns to the proteins dataframe
    
    Returns:
    -------
    df : pandas.DataFrame
        The data from the OPM database, including protein types,
        classes, superfamilies, families, species, and localization
    """
    
    # First we load all of the csv files into memory as pandas dataframes
    # proteins is the main table we are interested in
    proteins = pd.read_csv(get_path('protein.csv'), sep=';')

    # types, classes, families, and membranes will all become
    # dicts to interpret their various id's and turn them into names
    types = pd.read_csv(get_path('type.csv'), sep=';')
    classes = pd.read_csv(get_path('class.csv'), sep=';')
    families = pd.read_csv(get_path('family.csv'), sep=';')
    membranes = pd.read_csv(get_path('membrane.csv'), sep=';')

    # any given family superfamilies and species
    classifications = pd.read_csv(get_path('classification.csv'), sep=';')
    superfamilies = pd.read_csv(get_path('superfamily.csv'), sep=';')
    species = pd.read_csv(get_path('species.csv'), sep=';')

    # Next we make a series of dictionaries to translate:
    # family id --> family name
    # family id --> class number
    # family id --> type id
    # family id --> family tcdb code
    # family id --> family pfam code
    # family id --> superfamily_id
    family_names = dict(families[['id','name']].values)
    family_tcdbs = dict(families[['id','tcdb']].values)
    family_pfams = dict(families[['id','pfam']].values)
    
    # family id --> superfamily id
    # family id --> class id
    # family id --> type id
    family_to_superfam = dict(classifications[['family_id','superfamily_id']].values)
    family_to_class = dict(classifications[['family_id','class_id']].values)
    family_to_type = dict(classifications[['family_id','type_id']].values)

    # superfamily id --> superfamily name
    # superfamily id --> superfamily tcdb code
    # superfamily id --> superfamily pfam code
    superfamily_names = dict(superfamilies[['id','name']].values)
    superfamily_tcdbs = dict(superfamilies[['id','tcdb']].values)
    superfamily_pfams = dict(superfamilies[['id','pfam']].values)
    
    # class id --> class name
    # type id --> type name
    # species id --> species name
    # membrane id --> memrane name
    # memrane id --> membran abbreviation
    class_names = dict(classes[['id','name']].values)
    type_names = dict(types[['id','name']].values)
    species_names = dict(species[['id','name']].values)
    membrane_names = dict(membranes[['id','name']].values)
    membrane_abbr = dict(membranes[['id','abbreviation']].values)

    # now we use the dics from above to create new columns in the
    # proteins data frame containing the actual names of the
    # species, membrane, family, superfamily, etc
    proteins['species'] = proteins.species_id.replace(species_names)
    proteins['membrane'] = proteins.membrane_id.replace(membrane_names)
    proteins['membrane_abbr'] = proteins.membrane_id.replace(membrane_abbr)
    proteins['family'] = proteins.family_id.replace(family_names)
    proteins['family_pfam'] = proteins.family_id.replace(family_pfams)
    proteins['family_tcdb'] = proteins.family_id.replace(family_tcdbs)
    proteins['superfamily'] = proteins.family_id.replace(family_to_superfam).replace(superfamily_names)
    proteins['superfamily_tcdb'] = proteins.family_id.replace(family_to_superfam).replace(superfamily_tcdbs)
    proteins['superfamily_pfam'] = proteins.family_id.replace(family_to_superfam).replace(superfamily_pfams)
    proteins['class'] = proteins.family_id.replace(family_to_class).replace(class_names)
    proteins['type'] = proteins.family_id.replace(family_to_type).replace(type_names)
    
    return proteins
    
# load the data from excel files located in this directory
def load_excell_data():
    """
    Deprecated: Load the OPM database into memory as a pandas data frame.
    The protein data was downloaded from the OPM database
    as a MySQL dump file:
    http://opm.phar.umich.edu/OPM-2016-10-10.sql
    
    The data is was then converted to an excel file stored locally:
    "pdb-search/data/OPM_data_from_MySQL.xlsx"
    
    Returns:
    -------
    df : pandas.DataFrame
        The data from the OPM database, including protein types,
        classes, superfamilies, families, species, and localization
    """
    return pd.read_excel(get_path("OPM_data_from_MySQL.xlsx"), "Sheet1")

df_proteins = load_data()

# Step 2: Exploratory Graph

In [10]:
df_proteins.head(2) # the full dataframe

,id,family_id,species_id,membrane_id,pdbid,name,resolution,topology,thickness,thicknesserror,...,membrane,membrane_abbr,family,family_pfam,family_tcdb,superfamily,superfamily_tcdb,superfamily_pfam,class,type
0,1,35,9,3,1qjp,"Outer membrane protein A (OMPA), disordered loops",1.65,A in,25.4,1.5,...,Bacterial Gram-negative outer membrane,Bact. Gram-neg outer,OmpA family,PF01389,NaN,"OmpA-OmpF porin family (n=8,S=10)",1.B.6,NaN,Beta-barrel transmembrane,Transmembrane
1,2,390,9,3,1qj8,Outer membrane protein X (OMPX),1.90,A in,23.6,2.8,...,Bacterial Gram-negative outer membrane,Bact. Gram-neg outer,Enterobacterial Ail/Lom protein,PF06316,NaN,"OmpA-OmpF porin family (n=8,S=10)",1.B.6,NaN,Beta-barrel transmembrane,Transmembrane


In [ ]:
# initial database explore graph 


## Display Clicked Protein

In [3]:
clicked_protein = "7PRN"     # fetch clicked protein
view = nv.show_pdbid(clicked_protein) # access PDB online, and create cartoon
view # display
# TO DO: add PDB_ID, Protein_Name, type, class (from the OPM database, as metadata when hovering) 
# TO DO: add iframe to embed this demo display

A Jupyter Widget

# Step 3: Query 

In [29]:
#option 1: send the whole SQL query to sql_query()

qr_text ="""
    SELECT *
    FROM df_proteins
    WHERE (name LIKE '%channel%' OR name LIKE '%porin%') 
    AND NOT (name LIKE '%domain%' OR name LIKE '%monomer%') 
    AND (Class GLOB 'Beta-barrel transmembrane' )
    """
results = sql_query(qr_text)
results.head()

,id,family_id,species_id,membrane_id,pdbid,name,resolution,topology,thickness,thicknesserror,...,membrane,membrane_abbr,family,family_pfam,family_tcdb,superfamily,superfamily_tcdb,superfamily_pfam,class,type
0,11,43,9,3,3pox,Porin OmpF,2.00,A in,24.0,0.8,...,Bacterial Gram-negative outer membrane,Bact. Gram-neg outer,General Bacterial Porin (GBP),PF00267,1.B.1,"Trimeric porins (n=16,S=20)",None,None,Beta-barrel transmembrane,Transmembrane
1,13,43,8,3,2fgq,Anion-selective porin,1.45,A in,25.0,0.9,...,Bacterial Gram-negative outer membrane,Bact. Gram-neg outer,General Bacterial Porin (GBP),PF00267,1.B.1,"Trimeric porins (n=16,S=20)",None,None,Beta-barrel transmembrane,Transmembrane
2,14,88,30,3,2por,Porin,1.80,A in,23.4,0.6,...,Bacterial Gram-negative outer membrane,Bact. Gram-neg outer,Rhodobacter PorCa Porin (RPP),None,1.B.7,"Trimeric porins (n=16,S=20)",None,None,Beta-barrel transmembrane,Transmembrane
3,15,88,33,3,3prn,Porin,1.90,A in,23.2,0.5,...,Bacterial Gram-negative outer membrane,Bact. Gram-neg outer,Rhodobacter PorCa Porin (RPP),None,1.B.7,"Trimeric porins (n=16,S=20)",None,None,Beta-barrel transmembrane,Transmembrane
4,16,44,9,3,1af6,Maltoporin,2.40,A in,25.1,0.7,...,Bacterial Gram-negative outer membrane,Bact. Gram-neg outer,Malptoporin-like proteins,PF02264,1.B.3,"Sugar porins (n=18,S=22)",1.B.3,PF02264,Beta-barrel transmembrane,Transmembrane


In [31]:
#option 2: use a wrapper function that combines takes a table name and a selection as options

my_options = """
WHERE (name LIKE '%channel%' OR name LIKE '%porin%') 
AND NOT (name LIKE '%domain%' OR name LIKE '%monomer%') 
AND (Class GLOB 'Beta-barrel transmembrane' )
"""
results = make_sql("df_proteins","pdbid",my_options)
results
#print (sql_query("SELECT * FROM df_proteins LIMIT 10;").head())

,pdbid
0,3pox
1,2fgq
2,2por
3,3prn
4,1af6
5,2mpr
6,1a0s
7,1uun
8,1pho
9,2j1n


# Step 4: Query-Based Graph

# Step 5: 2D Informative Statistic